<a href="https://colab.research.google.com/github/Dhwanigoyal/sampling-ucs654/blob/main/SamplingUCS654.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load the dataset
data = pd.read_csv("Creditcard_data.csv")
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,1
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
data['Class'].value_counts()


,count
Class,
0,763
1,9


In [ ]:
from imblearn.over_sampling import SMOTE

X = data.drop('Class', axis=1)
y = data['Class']

smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X, y)

# Verify the balancing
pd.Series(y_balanced).value_counts()


,count
Class,
0,763
1,763


In [ ]:
from sklearn.utils import resample

sample1 = resample(data, replace=True, n_samples=100, random_state=1)
sample2 = resample(data, replace=True, n_samples=200, random_state=2)
sample3 = resample(data, replace=True, n_samples=300, random_state=3)
sample4 = resample(data, replace=True, n_samples=400, random_state=4)
sample5 = resample(data, replace=True, n_samples=500, random_state=5)


In [ ]:
import numpy as np

def random_sampling(data, size):
    return data.sample(n=size, random_state=42)

def stratified_sampling(data, size):
    from sklearn.model_selection import train_test_split
    return train_test_split(data, test_size=size/len(data), stratify=data['Class'])[0]

def systematic_sampling(data, size):
    interval = len(data) // size
    indices = np.arange(0, len(data), interval)
    return data.iloc[indices]

def cluster_sampling(data, num_clusters):
    clusters = np.array_split(data, num_clusters)
    chosen_cluster = clusters[0]
    return chosen_cluster

def oversampling(data, size):
    from imblearn.over_sampling import RandomOverSampler
    ros = RandomOverSampler(random_state=42)
    X, y = ros.fit_resample(data.drop('Class', axis=1), data['Class'])
    return pd.concat([pd.DataFrame(X), pd.DataFrame(y, columns=['Class'])], axis=1)

sampling_functions = {
    "Random Sampling": random_sampling,
    "Stratified Sampling": stratified_sampling,
    "Systematic Sampling": systematic_sampling,
    "Cluster Sampling": lambda data, size: cluster_sampling(data, 5), # Example: 5 clusters
    "Oversampling": oversampling
}


# Apply sampling techniques
sampled_datasets = {name: func(data, 300) for name, func in sampling_functions.items()}


/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC(),
    'KNN': KNeighborsClassifier()
}

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

for name, model in models.items():
    X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.3, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"{name} Accuracy: {accuracy_score(y_test, y_pred)}")


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy: 0.9061135371179039
Decision Tree Accuracy: 0.982532751091703
Random Forest Accuracy: 0.9934497816593887
SVM Accuracy: 0.6768558951965066
KNN Accuracy: 0.8471615720524017


In [ ]:
results = []

for sampling_name, sampled_data in sampled_datasets.items():
    print(f"\nEvaluating models on {sampling_name} data...")
    X_sample = sampled_data.drop('Class', axis=1)
    y_sample = sampled_data['Class']

    X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.3, random_state=42)

    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        results.append((sampling_name, model_name, accuracy))
        print(f"Model: {model_name}, Accuracy: {accuracy:.2f}")

results_df = pd.DataFrame(results, columns=["Sampling Technique", "Model", "Accuracy"])
print("\nResults Summary:")
print(results_df)


results_df.to_csv("Sampling_Assignment_Results.csv", index=False)



Evaluating models on Random Sampling data...
Model: Logistic Regression, Accuracy: 0.94
Model: Decision Tree, Accuracy: 0.92
Model: Random Forest, Accuracy: 0.94
Model: SVM, Accuracy: 0.94
Model: KNN, Accuracy: 0.94

Evaluating models on Stratified Sampling data...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Model: Logistic Regression, Accuracy: 0.99
Model: Decision Tree, Accuracy: 0.98
Model: Random Forest, Accuracy: 0.99
Model: SVM, Accuracy: 0.99
Model: KNN, Accuracy: 0.99

Evaluating models on Systematic Sampling data...
Model: Logistic Regression, Accuracy: 1.00
Model: Decision Tree, Accuracy: 1.00


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model: Random Forest, Accuracy: 1.00
Model: SVM, Accuracy: 1.00
Model: KNN, Accuracy: 1.00

Evaluating models on Cluster Sampling data...
Model: Logistic Regression, Accuracy: 1.00
Model: Decision Tree, Accuracy: 1.00
Model: Random Forest, Accuracy: 1.00
Model: SVM, Accuracy: 1.00
Model: KNN, Accuracy: 1.00

Evaluating models on Oversampling data...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model: Logistic Regression, Accuracy: 0.91
Model: Decision Tree, Accuracy: 0.99
Model: Random Forest, Accuracy: 1.00
Model: SVM, Accuracy: 0.69
Model: KNN, Accuracy: 0.98

Results Summary:
     Sampling Technique                Model  Accuracy
0       Random Sampling  Logistic Regression  0.944444
1       Random Sampling        Decision Tree  0.922222
2       Random Sampling        Random Forest  0.944444
3       Random Sampling                  SVM  0.944444
4       Random Sampling                  KNN  0.944444
5   Stratified Sampling  Logistic Regression  0.985915
6   Stratified Sampling        Decision Tree  0.978873
7   Stratified Sampling        Random Forest  0.985915
8   Stratified Sampling                  SVM  0.985915
9   Stratified Sampling                  KNN  0.985915
10  Systematic Sampling  Logistic Regression  1.000000
11  Systematic Sampling        Decision Tree  1.000000
12  Systematic Sampling        Random Forest  1.000000
13  Systematic Sampling                  

In [ ]:
# Group results by model and find the sampling technique with the highest accuracy
best_techniques = results_df.loc[results_df.groupby('Model')['Accuracy'].idxmax()]

print("Best Sampling Techniques for Each Model:")
print(best_techniques)


Best Sampling Techniques for Each Model:
     Sampling Technique                Model  Accuracy
11  Systematic Sampling        Decision Tree       1.0
14  Systematic Sampling                  KNN       1.0
10  Systematic Sampling  Logistic Regression       1.0
12  Systematic Sampling        Random Forest       1.0
13  Systematic Sampling                  SVM       1.0
